In [20]:
!pip install langchain
!pip install langchain_core
!pip install langchain_openai
!pip install langchain_community
!pip install chromadb
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 7.7 MB/s eta 

In [7]:
import os
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
    create_structured_chat_agent
)
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [10]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("openai_api_key")

def get_current_time(*args, **kwargs):
  import datetime
  now = datetime.datetime.now()
  return now.strftime("%I:%M %p")

def search_wikipedia(query):
  from wikipedia import summary
  try:
    return summary(query, sentences=2)
  except:
    return "No information found"

tools = [
    Tool(
        name="Time",
        func=get_current_time,
        description="Useful for when you need to know the curretn time",
    ),
    Tool(
        name="Wikipedia",
        func=search_wikipedia,
        description="Get wikipedia summary",
    ),
]

prompt = hub.pull("hwchase17/structured-chat-agent")

llm = ChatOpenAI(model="gpt-4o")

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

agent = create_structured_chat_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
)

initial_message = "You are an AI assistant that can provide helpful answers using available tools. \nIf you are unable to answer, you can use the following tools: Time and Wikipedia."

memory.chat_memory.add_message(SystemMessage(content=initial_message))

while True:
  user_input = input("User: ")
  if user_input.lower() == "exit":
    break

  memory.chat_memory.add_message(HumanMessage(content=user_input))
  response = agent_executor.invoke({"input": user_input})
  print("Bot:", response["output"])
  memory.chat_memory.add_message(AIMessage(content=response["output"]))


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


User: can you get me the details about the Odyssey movie directed ny Christopher Nolan?


> Entering new AgentExecutor chain...
{
  "action": "Wikipedia",
  "action_input": "The Odyssey movie directed by Christopher Nolan"
}The Odyssey is an upcoming epic action fantasy film written, produced, and directed by Christopher Nolan as an adaptation of the eponymous epic poem by Homer, produced by Nolan's production company Syncopy and distributed by Universal Pictures. The film stars Matt Damon as Odysseus, the Ancient Greek king of Ithaca, and follows his journey home after the Trojan War to reunite with his wife, Penelope.{
  "action": "Final Answer",
  "action_input": "The Odyssey is an upcoming epic action fantasy film written, produced, and directed by Christopher Nolan. It is an adaptation of the epic poem by Homer, produced by Nolan's production company Syncopy and distributed by Universal Pictures. The film stars Matt Damon as Odysseus, the Ancient Greek king of Ithaca, and follows 

ToolException: Too many arguments to single-input tool Time.
                Consider using StructuredTool instead. Args: []

In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

Mounted at /content/drive


In [22]:
current_dir = os.getcwd()
db_dir = os.path.join(current_dir, "drive/MyDrive/Colab Notebooks/db")
persistent_directory = os.path.join(db_dir, "chroma_db")

if os.path.exists(persistent_directory):
    db = Chroma(persist_directory=persistent_directory, embedding_function=None)
else:
    raise FileNotFoundError(
        f"{persistent_directory} does not exist."
    )

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

llm = ChatOpenAI(model="gpt-4o")

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

react_docstore_prompt = hub.pull("hwchase17/react")

tools = [
    Tool(
        name="Answer Question",
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input": input, "chat_history": kwargs.get("chat_history", [])}
        ),
        description="useful for when you need to answer questions about the context",
    )
]

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt,
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True,
)

chat_history = []
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})
    print(f"AI: {response['output']}")

    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response["output"]))

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You: where did Odysseus fight?


> Entering new AgentExecutor chain...
Odysseus is a character from Greek mythology who fought in the Trojan War. The Trojan War was a legendary conflict between the early Greeks and the people of Troy, which is located in what is now Turkey. Odysseus is best known for his long journey home from this war, as detailed in Homer's epic poem, "The Odyssey." 

Final Answer: Odysseus fought in the Trojan War, which took place in the ancient city of Troy.

> Finished chain.
AI: Odysseus fought in the Trojan War, which took place in the ancient city of Troy.
You: did he win the war?


> Entering new AgentExecutor chain...
To determine if "he" won the war, I need more context about the specific individual or war being referred to. 

Action: Answer Question
Action Input: Who is "he" and which war are you referring to?
{'input': 'Who is "he" and which war are you referring to?\n', 'chat_history': [], 'context': [Document(metadata={'source': '/content/drive/MyDrive/